In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = "/content/drive/MyDrive/barbechli.xlsx"
df = pd.read_excel(file_path)

df_cleaned = df.drop_duplicates(keep=False)

removed_rows = len(df) - len(df_cleaned)
print(f"Nombre de lignes supprimées : {removed_rows}")

df_cleaned.to_excel("/content/drive/MyDrive/fichier_sans_doublons.xlsx", index=False)
print("Les doublons ont été supprimés et le nouveau fichier a été enregistré sous 'fichier_sans_doublons.xlsx'.")

Nombre de lignes supprimées : 712
Les doublons ont été supprimés et le nouveau fichier a été enregistré sous 'fichier_sans_doublons.xlsx'.


In [ ]:
import pandas as pd
import re

input_excel = '/content/drive/MyDrive/fichier_sans_doublons.xlsx'
output_excel = '/content/drive/MyDrive/barbechli2.xlsx'

try:
    df = pd.read_excel(input_excel)
    print("Fichier Excel lu avec succès.")
except Exception as e:
    raise ValueError(f"Erreur lors de la lecture du fichier Excel : {e}")

def extract_entity(url):
    if pd.isna(url):
        return None
    match = re.search(r'logo-([a-zA-Z0-9_]+)\.jpg', url)
    if match:
        return match.group(1).replace('_', ' ').title()
    return None

df['Company'] = df['Company-path'].apply(extract_entity)

def extract_characteristics(text):
    if pd.isna(text):
        return {
            'Type': None,
            'Model': None,
            'Processor Brand': None,
            'Processor': None,
            'RAM': None,
            'Storage': None,
            'GPU': None,
            'Screen': None,
            'Color': None,
            'OS': None
        }

    characteristics = {
        'Type': None,
        'Model': None,
        'Processor Brand': None,
        'Processor': None,
        'RAM': None,
        'Storage': None,
        'GPU': None,
        'Screen': None,
        'Color': None,
        'OS': None
    }

    # Extraire le type (PC Portable, Écran Gaming, etc.)
    type_match = re.search(r'(PC Portable|Écran Gaming|Pc Portable|PC PORTABLE|Ecran Gaming|Ecran|Portable|Moniteur)', text, re.IGNORECASE)
    if type_match:
        characteristics['Type'] = type_match.group(1)

    # Extraire le modèle (tout ce qui suit le type jusqu'au processeur ou à la RAM)
    model_match = re.search(r'(?:PC Portable|Écran Gaming|Pc Portable|PC PORTABLE|Ecran Gaming|Ecran|Portable|Moniteur)\s+(.*?)\s+(Intel|AMD|Apple|Celeron|Ryzen|N\d+|N\d+-\d+[a-zA-Z]*|Ultra \d+|M\d+|R\d+|\d+\s*Go)', text, re.IGNORECASE)
    if model_match:
        characteristics['Model'] = model_match.group(1).strip()

    # Extraire la marque du processeur (Intel, AMD, etc.)
    processor_brand_match = re.search(r'(Intel|AMD|Apple|Celeron|Ryzen|N\d+|N\d+-\d+[a-zA-Z]*|Ultra \d+|M\d+|R\d+)', text, re.IGNORECASE)
    if processor_brand_match:
        characteristics['Processor Brand'] = processor_brand_match.group(1)

    # Extraire le processeur (iX, Ryzen X, Celeron X, etc.)
    processor_match = re.search(r'([iI]\d+|Ryzen \d+|Celeron \w+|N\d+|N\d+-\d+[a-zA-Z]*|Ultra \d+|M\d+|R\d+)', text)
    if processor_match:
        characteristics['Processor'] = processor_match.group(1)

    # Extraire la RAM (X Go)
    ram_match = re.search(r'(\d+\s*Go)', text, re.IGNORECASE)
    if ram_match:
        characteristics['RAM'] = ram_match.group(1)

    # Extraire le stockage (X Go SSD, X TB SSD, etc.)
    storage_match = re.search(r'(\d+\s*Go\s*SSD|\d+\s*TB\s*SSD|\d+\s*Go|\d+\s*TB|\d+\s*Go\s*HDD)', text, re.IGNORECASE)
    if storage_match:
        characteristics['Storage'] = storage_match.group(1)

    # Extraire le GPU (RTX X, GTX X)
    gpu_match = re.search(r'(RTX\s*\d+|GTX\s*\d+)', text, re.IGNORECASE)
    if gpu_match:
        characteristics['GPU'] = gpu_match.group(1)

    # Extraire les informations sur l'écran (taille, résolution, etc.)
    screen_match = re.search(r'(\d+["”]|\d+\s*cm|\d+\s*inch)', text, re.IGNORECASE)
    if screen_match:
        characteristics['Screen'] = screen_match.group(1)

    # Extraire la couleur (Gris, Silver, Bleu, etc.)
    color_match = re.search(r'(Gris|Silver|Bleu|Noir|Gold|Rose|Rouge|Vert)', text, re.IGNORECASE)
    if color_match:
        characteristics['Color'] = color_match.group(1)

    # Extraire le système d'exploitation (Windows 11, etc.)
    os_match = re.search(r'(Windows\s*\d+|MacOS|Linux)', text, re.IGNORECASE)
    if os_match:
        characteristics['OS'] = os_match.group(1)

    return characteristics

# Appliquer la fonction pour extraire les caractéristiques
characteristics = df['Description'].apply(extract_characteristics).apply(pd.Series)

# Concaténer les caractéristiques avec le DataFrame original
df = pd.concat([df, characteristics], axis=1)

# Enregistrer le DataFrame transformé dans un nouveau fichier Excel
try:
    df.to_excel(output_excel, index=False)  # Enregistrer en tant que fichier Excel
    print(f"Le fichier traité a été enregistré sous {output_excel}")
except Exception as e:
    raise ValueError(f"Erreur lors de l'enregistrement du fichier Excel : {e}")

Fichier Excel lu avec succès.
Le fichier traité a été enregistré sous /content/drive/MyDrive/barbechli2.xlsx


In [ ]:
import pandas as pd

# Chemin du fichier
file_path = "/content/drive/MyDrive/barbechli2.xlsx"

# Charger le fichier Excel
df = pd.read_excel(file_path)

# Modifier la colonne 'entity' en supprimant "Online" si "Technopro Online" est trouvé
df["Company"] = df["Company"].replace("Technopro Online", "Technopro")

df.to_excel(file_path, index=False)

print("Les modifications ont été appliquées directement dans 'barbechli2.xlsx'.")

Les modifications ont été appliquées directement dans 'barbechli2.xlsx'.


In [ ]:
import pandas as pd

# 📌 Chemin du fichier d'entrée (Excel) et sortie (Excel)
input_file = "/content/drive/MyDrive/barbechli2.xlsx"

# ✅ Charger le fichier Excel
df = pd.read_excel(input_file)

# ✅ Nettoyage des colonnes Price et Price 2 (suppression des espaces et conversion en float)
df["Price"] = df["Price"].astype(str).str.replace(" ", "").replace("", None).astype(float)
df["Price 2"] = df["Price 2"].astype(str).str.replace(" ", "").replace("", None).astype(float)

# ✅ Nettoyage de Discount percentage (suppression de "%")
df["Discount percentage"] = df["Discount percentage"].astype(str).str.replace("%", "").replace("", None).astype(float)

# ✅ Sauvegarde en fichier Excel
df.to_excel(input_file, index=False)

print(f"✅ Fichier nettoyé enregistré dans : {input_file}")

✅ Fichier nettoyé enregistré dans : /content/drive/MyDrive/barbechli2.xlsx


In [ ]:
import pandas as pd

# 📌 Chemin du fichier d'entrée (Excel) et sortie (CSV)
input_file = "/content/drive/MyDrive/barbechli2.xlsx"  # Fichier Excel en entrée
output_csv = "/content/drive/MyDrive/barbechli2.csv"  # Fichier CSV en sortie

# ✅ Charger le fichier Excel
df = pd.read_excel(input_file)

# ✅ Sauvegarder en fichier CSV
df.to_csv(output_csv, index=False)

print(f"✅ Fichier Excel converti et enregistré dans : {output_csv}")

✅ Fichier Excel converti et enregistré dans : /content/drive/MyDrive/barbechli2.csv


In [ ]:
pip install psycopg2

In [ ]:
pip freeze > requeriments.txt

In [ ]:
import pandas as pd

# Lire le fichier CSV
df = pd.read_csv(r"/content/drive/MyDrive/barbechli2.csv")

# Supprimer les doublons basés sur la colonne 'id'
df = df.drop_duplicates(subset=['ID'])

# Sauvegarder le fichier nettoyé
df.to_csv(r"/content/drive/MyDrive/barbechli2.csv", index=False)

In [ ]:
import psycopg2
import pandas as pd

# Chemin du fichier CSV
csv_file = r"C:\Users\messa\OneDrive\Documents\Scrap\barbechli2.csv"

# Connexion à PostgreSQL
conn = psycopg2.connect(
    dbname="Barbechli",
    user="postgres",
    password="7y4bH5Cs",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Lire le fichier CSV avec pandas
df = pd.read_csv(csv_file)

# Insérer les données dans la table
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO products (
            company_path, description, price, price_2, currency, discount_percentage,
            company, type, model, processor_brand, processor, ram, storage, gpu, screen, color, os
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, tuple(row))

# Valider les changements et fermer la connexion
conn.commit()
cursor.close()
conn.close()

print("✅ Données CSV importées avec succès dans PostgreSQL.")

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (::1), port 5432 failed: Cannot assign requested address
	Is the server running on that host and accepting TCP/IP connections?
